In [ ]:
#Environment set-up and libraries

#Base libraries
import numpy as np
import random
import torch

#Plotting libraries
%matplotlib inline
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go

#Utilities libraries
from glob import glob 
import os

import open3d as o3d

In [ ]:
# We create a function that load and normalize a point cloud tile
def cloud_loader(tile_name, features_used):
  cloud_data = np.loadtxt(tile_name).transpose()

  min_f=np.min(cloud_data,axis=1)
  mean_f=np.mean(cloud_data,axis=1)

  features=[]
  if 'xyz' in features_used:
    n_coords = cloud_data[0:3]
    n_coords[0] -= mean_f[0]
    n_coords[1] -= mean_f[1]
    n_coords[2] -= min_f[2]
    features.append(n_coords)
  if 'rgb' in features_used:
    colors = cloud_data[3:6]/255
    features.append(colors)
  if 'i' in features_used:
    # n_intensity = cloud_data[-2] - min_f[-2]
    IQR = np.quantile(cloud_data[-2],0.75)-np.quantile(cloud_data[-2],0.25)
    n_intensity = ((cloud_data[-2] - np.median(cloud_data[-2])) / IQR)
    n_intensity -= np.min(n_intensity)
    # intensity[np .where(intensity > 5000)]=5000
    features.append(n_intensity)
  
  gt = cloud_data[-1]
  gt = torch.from_numpy(gt).long()
  cloud_data = torch.from_numpy(np.vstack(features))
  return cloud_data, gt

In [ ]:
def load_file(file_name):
    print(file_name)

    if file_name.endswith(".las") or file_name.endswith(".laz"):
        print("[INFO] .las (.laz) file loading")
        try:
            # import lidar .las data and assign to variable
            pcd = laspy.read(file_name)
            # examine the available features for the lidar file we have read
            # list(las.point_format.dimension_names)
            #
            # set(list(las.classification))

            # Creating, Filtering, and Writing Point Cloud Data
            # To create 3D point cloud data, we can stack together with the X, Y, and Z dimensions, using Numpy like this.
            point_data = np.stack([pcd.X, pcd.Y, pcd.Z], axis=0).transpose((1, 0))
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_data)
            # self.points = point_data
            if pcd is not None:
                print("[Info] Successfully read", file_name)

                # Point cloud
                return pcd

        except Exception:
            print(".las, .laz file load failed")

    elif file_name.endswith(".e57"):
        print("[INFO] .e57 file loading")
        try:
            e57_file = pye57.E57(file_name)

            # other attributes can be read using:
            data = e57_file.read_scan(0)

            # 'data' is a dictionary with the point types as keys
            # assert isinstance(data["cartesianX"], np.ndarray)
            # assert isinstance(data["cartesianY"], np.ndarray)
            # assert isinstance(data["cartesianZ"], np.ndarray)

            point_xyz = np.stack([data["cartesianX"], data["cartesianY"], data["cartesianZ"]]).transpose((1, 0))
            # points_rgb = [data["colorRed"], data["colorGreen"], data["colorBlue"]]
            # points_intensity = data["intensity"]

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_xyz)
            # self.points = o3d.utility.Vector3dVector(point_xyz)
            # self.points = point_xyz
            # self.pcd.colors = o3d.utility.Vector3dVector(points_rgb)
            # self.pcd.colors[:, 0] = points_intensity
            print("[Info] Successfully read", file_name)
            return pcd

        except Exception:
            print(".e57 file load failed")

    elif file_name.endswith(".ply"):
        pcd = o3d.io.read_point_cloud(file_name)
        points_xyz = np.asarray(pcd.points)
        #self.pcd = o3d.geometry.PointCloud() # No need to do that already a PointCloud
        pcd.points = o3d.utility.Vector3dVector(points_xyz)
        # self.points = points_xyz
        if pcd is not None:
            print("[Info] Successfully read", file_name)
            # Point cloud
            return pcd
    elif file_name.endswith(".pts"):
        points_np = []
        try:
            with open(file_name, "r") as f:                
                for line in f:
                    if len(line.split()) == 4:
                        x, y, z, i = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z), int(float(i))])
                        #print('point', len(points_np))
                    elif len(line.split()) == 3:
                        x, y, z = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z)])
                        #print('point', len(points_np))
                    elif len(line.split()) == 5:
                        x, y, z, i, zeroes_v = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z), int(float(i))])
                        #print('point', len(points_np))
                    else:
                        print("[Info] The file has unregistered format")
            print('point', len(points_np))
            print('loop end')
            points_arr = np.array(points_np).transpose()
            print(len(points_arr))
            point_xyz = points_arr[:3].transpose()
            # points_intensity = points_arr[3]
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_xyz)
            if pcd is not None:
                print("[Info] Successfully read", file_name)
                # Point cloud
                return pcd

        except Exception:
            print("[Info] Unsuccessfully read .pts file", file_name)

    else:
        pcd = None
        geometry_type = o3d.io.read_file_geometry_type(file_name)
        print(geometry_type)

        mesh = None
        if geometry_type & o3d.io.CONTAINS_TRIANGLES:
            mesh = o3d.io.read_triangle_model(file_name)
        if mesh is None:
            print("[Info]", file_name, "appears to be a point cloud")
            cloud = None
            try:
                cloud = o3d.io.read_point_cloud(file_name)
                # print(type(cloud))
            except Exception:
                print("[Info] Unknown filename", file_name)
            if cloud is not None:
                print("[Info] Successfully read", file_name)

                if not cloud.has_normals():
                    cloud.estimate_normals()
                cloud.normalize_normals()
                pcd = cloud
                #self.points = cloud.points
                pcd.points = o3d.utility.Vector3dVector(cloud.points)
            else:
                print("[WARNING] Failed to read points", file_name)

        if pcd is not None or mesh is not None:
            try:
                if mesh is not None:
                    # Triangle model
                    pass #_scene.scene.add_model("__model__", mesh)
                else:
                    # Point cloud
                    return pcd

            except Exception as e:
                print(e)

In [ ]:
filepath_static = "/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/Highway Sample Set 1/"
filename = "18MAY23S1_ - 0014 - F5.pts"
filepath = filepath_static + filename
pc = load_file(filepath)
    


In [ ]:
print('median\n',np.median(pc.points,axis=0))
print('std\n',np.std(pc.points,axis=0))
print('min\n',np.min(pc.points,axis=0))
print('max\n',np.max(pc.points,axis=0))

In [ ]:
#Computing the mean and the min of a data tile
cloud_data=np.asarray(pc.points).transpose()
min_f=np.min(cloud_data,axis=1)
mean_f=np.mean(cloud_data,axis=1)
print(min_f)
print(mean_f)

In [ ]:
#Normalizing coordinates
n_coords = cloud_data[0:3]
n_coords[0] -= mean_f[0]
n_coords[1] -= mean_f[1]
n_coords[2] -= min_f[2]
print(n_coords)

In [ ]:
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(pc.points)
#pcd.colors=o3d.utility.Vector3dVector((np.array(pc)[3:6]).transpose())

o3d.visualization.draw_plotly([pcd],point_sample_factor=0.5, width=600, height=400)

In [ ]:
def visualize_input_tile(tile_name, classes, features_used='xyzi', sample_size=None, figsize=900, width=None, height=None, pointsize=3):
  # Load the tile 
  cloud, labels = cloud_loader(tile_name, features_used)

  #vizualisation sampling
  if sample_size != None:
    idx = np.random.choice(np.arange(cloud.size()[1]), sample_size, replace=False)
    cloud = cloud[:,idx]
    labels = labels[idx]
  # Initialize the class names and colors
  class_names = classes
  print(f"classes used are: {class_names}")
  class_colors = ['black', 'wheat', 'lightgreen', 'pink','cyan','green','red']

  # 3D visualization modes
  modes = {'name': [], 'key': [], 'num_traces': []}

  # Recover point coordinates and intensity
  n_points = cloud.shape[1]
  x = cloud[0, :]
  y = cloud[1, :]
  z = cloud[2, :]
  intensity = cloud[3, :]

  # Round to the cm for cleaner hover info
  x = (x * 100).round() / 100
  y = (y * 100).round() / 100
  z = (z * 100).round() / 100

  # Prepare figure
  width = width if width and height else figsize
  height = height if width and height else int(figsize / 2)
  margin = int(0.02 * min(width, height))
  layout = go.Layout(
    width=width,
    height=height,
    scene=dict(aspectmode='data', ),  # preserve aspect ratio
    margin=dict(l=margin, r=margin, b=margin, t=margin),
    uirevision=True)
  fig = go.Figure(layout=layout)
  initialized_visibility = False

  # Draw a trace for each class present in the sample
  n_gt_traces = 0

  for label in np.unique(labels):
    indices = np.where(label == labels)[0]

    fig.add_trace(
      go.Scatter3d(
        name=class_names[label] if class_names else f"Class {label}",
        x=x[indices],
        y=y[indices],
        z=z[indices],
        mode='markers',
        marker=dict(
          size=pointsize,
          color=class_colors[label] if class_colors else None, ),
        visible=not initialized_visibility, ))
    n_gt_traces += 1  # keep track of the number of traces

  modes['name'].append('Ground Truth')
  modes['key'].append('gt')
  modes['num_traces'].append(n_gt_traces)
  initialized_visibility = True

  # Draw a trace for the intensity-colored cloud
  fig.add_trace(
    go.Scatter3d(
      name='Intensity',
      x=x,
      y=y,
      z=z,
      mode='markers',
      marker=dict(
        size=pointsize,
        color=intensity, 
        colorscale='hot',
        cmin=0,
        cmax=10, ),
      hoverinfo='x+y+z',
      showlegend=False,
      visible=not initialized_visibility, ))
  modes['name'].append('Intensity')
  modes['key'].append('intensity')
  modes['num_traces'].append(1)
  initialized_visibility = True

    # Unannotated trace
  indices = np.where(labels == 0)[0]

  fig.add_trace(
    go.Scatter3d(
      name='Unannotated',
      x=x[indices],
      y=y[indices],
      z=z[indices],
      mode='markers',
      marker=dict(
        size=pointsize,
        color='rgb(0, 0, 0)', ),
      visible=not initialized_visibility, ))

  # Traces visibility for interactive point cloud coloring
  def trace_visibility(mode):
    visibilities = np.array([d.visible for d in fig.data], dtype='bool')

    # Traces visibility for interactive point cloud coloring
    i_mode = modes['key'].index(mode)
    a = sum(modes['num_traces'][:i_mode])
    b = sum(modes['num_traces'][:i_mode + 1])
    n_traces = sum(modes['num_traces'])

    visibilities[:n_traces] = False
    visibilities[a:b] = True

    return [{"visible": visibilities.tolist()}]

  # Create the buttons that will serve for toggling trace visibility
  updatemenus = [
    dict(
      buttons=[
        dict(label=name, method='update', args=trace_visibility(key))
        for name, key in zip(modes['name'], modes['key'])],
      pad={'r': 10, 't': 10},
      showactive=True,
      type='dropdown',
      direction='right',
      xanchor='left',
      x=0.02,
      yanchor='top',
      y=1.02, ),
  ]
  fig.update_layout(updatemenus=updatemenus)

  # Place the legend on the left
  fig.update_layout(
    legend=dict(
      yanchor="middle",
      y=0.5,
      xanchor="right",
      x=0.99))

  # Hide all axes and no background
  fig.update_layout(
    scene=dict(
      xaxis_title='',
      yaxis_title='',
      zaxis_title='',
      xaxis=dict(
        autorange=True,
        showgrid=False,
        ticks='',
        showticklabels=False,
        backgroundcolor="rgba(0, 0, 0, 0)"
      ),
      yaxis=dict(
        autorange=True,
        showgrid=False,
        ticks='',
        showticklabels=False,
        backgroundcolor="rgba(0, 0, 0, 0)"
      ),
      zaxis=dict(
        autorange=True,
        showgrid=False,
        ticks='',
        showticklabels=False,
        backgroundcolor="rgba(0, 0, 0, 0)"
      )
    )
  )

  # Plot the figure
  fig.show(config={'displayModeBar': False})